# Ottenimento del modello

In [1]:
!mkdir models
%cd models
!wget https://grfia.dlsi.ua.es/primus/models/PrIMuS/Semantic-Model.zip
!unzip Semantic-Model.zip -d semantic

Cloning into 'tf-end-to-end'...
remote: Enumerating objects: 160, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 160 (delta 40), reused 60 (delta 32), pack-reused 86
Receiving objects: 100% (160/160), 402.02 KiB | 4.47 MiB/s, done.
Resolving deltas: 100% (72/72), done.
/content/tf-end-to-end
/content/tf-end-to-end/models
--2024-02-25 15:22:04--  https://grfia.dlsi.ua.es/primus/models/PrIMuS/Semantic-Model.zip
Resolving grfia.dlsi.ua.es (grfia.dlsi.ua.es)... 193.145.231.49
Connecting to grfia.dlsi.ua.es (grfia.dlsi.ua.es)|193.145.231.49|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 190886707 (182M) [application/zip]
Saving to: ‘Semantic-Model.zip’

Semantic-Model.zip  100%[===================>] 182.04M  7.28MB/s    in 22s     

2024-02-25 15:22:27 (8.15 MB/s) - ‘Semantic-Model.zip’ saved [190886707/190886707]

Archive:  Semantic-Model.zip
  inflating: semantic/semantic_model.data-00000-of-000